## **Setup**

In [1]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import pandas as pd

In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

In [4]:
from fastai.vision.all import *

In [5]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [6]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/LNDST/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/LNDST/Models/'
results_path='My Drive/AI For Good - AI Blitz 3/LNDST/Results/'
filename = 'train_images.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# **Functions**



In [7]:
def display_mask(imgnum):
  img=Image.open('/content/All_Data/Labels/train_gt/image_'+imgnum+'.png')
  img=np.asarray(img)*255
  display(Image.fromarray(np.uint8(img)))

# Unzip


In [8]:
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists('/content/All_Data/'):
  os.mkdir('/content/All_Data/')
  os.mkdir('/content/All_Data/Images/')
  # os.mkdir('/content/All_Data/Labels/')

  with ZipFile(f'{mount_path}/{competition_path}/test_images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/All_Data/Images/')
  # with ZipFile(f'{mount_path}/{competition_path}/train_gt.zip', 'r') as zip_ref:
  #   zip_ref.extractall('/content/All_Data/Labels/')
    

In [9]:
# imgnum=str(np.random.randint(1399))
# display_mask(imgnum)
# display(Image.open('/content/All_Data/Images/train_images/image_'+imgnum+'.jpg'))

In [10]:
testdir='/content/All_Data/Images/test_images/'

In [11]:
def get_msk(itmpath):
  return os.path.join(traindir,f'Labels/train_gt/{itmpath.stem}.png')
  
# test
# get_msk(Path('/content/All_Data/Images/train_images/image_10.jpg'))

Train the Model Zoo

In [12]:
import gc

In [13]:
# model_files=glob.glob(f'{os.path.join(mount_path,model_path)}*.pkl')
# print(model_files)

model_files=[
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0710_resnet18_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0723_resnet18_cv1.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0736_resnet18_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0749_xresnet18_cv0.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0803_xresnet18_cv1.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0816_xresnet18_cv2.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0830_resnet34_cv0.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0844_resnet34_cv1.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0857_resnet34_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0911_xresnet34_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0925_xresnet34_cv1.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/LNDST/Models/200901_0939_xresnet34_cv2.pkl']

In [14]:
imglst=[f'/content/All_Data/Images/test_images/image_{ii}.jpg' for ii in range(len(get_image_files(testdir)))]

In [ ]:
for model in model_files:
  learn = load_learner(model)

  model_name=os.path.basename(model[:-4])
  print(model_name)
  
  tst_dl=learn.dls.test_dl(imglst)

  learn.epoch=0
  preds = learn.tta(dl=tst_dl,n=4)[0]
  # preds = learn.tta(dl=tst_dl,n=2)[0]

  np.save(f'{mount_path}{results_path}{model_name}_test.npy', preds.numpy())

  del learn, preds
  gc.collect()
  torch.cuda.empty_cache()
  gc.collect()

200901_0710_resnet18_cv0


In [ ]:
# predsnp=preds.numpy()

In [ ]:
# def display_mask(img):
#   display(Image.fromarray(np.uint8(img*255)))

In [ ]:
# binpredsnp=np.argmax(predsnp,1)

In [ ]:
# binpredsnp.shape

In [ ]:
# ind=7
# print(imglst[ind])

# # display_mask(np.argmax(preds[ind],0))
# display_mask(binpredsnp[ind])
# # Image.open(np.array(imglst)[sort_inds][ind])
# Image.open(imglst[ind])